# FM Features
## Load Data

In [ ]:
import mne
import peakfrequency.peak_frequency as pf
from mne.channels.layout import _auto_topomap_coords as pos_from_raw
import numpy as np
import matplotlib.pyplot as plt

'''
fname = 'data/S2/S2_4chns.raw'
raw = mne.io.Raw(fname, preload=True)
data = raw.get_data()
fs = 500
channels = 4
samples = 900
data = data.T
'''
# Load data, set montage and remove bad channels as before
# Load data
path = "data/TF/"
raw = mne.io.read_raw_brainvision(path + "20191201_Cybathlon_TF_Session1_Block1.vhdr", preload=True)
# Set montage (location of channels)
raw.rename_channels({'O9': 'I1', 'O10': 'I2'})
montage = mne.channels.read_montage("standard_1005")
raw.set_montage(montage)
raw.rename_channels({'I1': 'O9', 'I2': 'O10'})
# Remove bad channels from analysis
raw.info['bads'] = ['F2', 'FFC2h', 'POO10h', 'O2']
picks = mne.pick_types(raw.info, eeg=True, stim=False, exclude='bads')

fs = int(1/raw.times[1])
# Create events from triggers
events = mne.events_from_annotations(raw)[0]
# Epoch data (cut up data into trials)
tmin = 1            # time in seconds after trigger the trial should start
tmax = tmin + 5     # time in seconds after trigger the trial should end
epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, preload=True, baseline=None, picks=picks)
data_lhand = epochs['1'].get_data()
data_rhand = epochs['2'].get_data()
nr_channels = data_rhand[0].shape[0]

## Optimizing Alpha Range

In [ ]:
def get_tripple(l, u, name):
    name = name+' ('+str(l)+' , '+str(u)+')'
    return float(l), float(u), name

def reset_band_bounds(bands, sub, obj, sup, l, u):
        bands[sub] = get_tripple(bands[sub][0], l, sub)
        bands[obj] = get_tripple(l, u, obj)
        bands[sup] = get_tripple(u, bands[sup][1], sup)
        return bands

def score_product(a, b):
    return a*b

def swap_score(score, best_score, lim, best_lim):
    if score > best_score:
        return score, lim
    else: 
        return best_score, best_lim

def optimize_alpha(data, bands = {'Delta': (get_tripple(1, 4,'Delta')), 'Theta': (get_tripple(4, 8,'Theta')), 'Alpha': (get_tripple(8, 12,'Alpha')), 'Beta': (get_tripple(12, 30, 'Beta')), 'Gamma': (get_tripple(30, 45,'Gamma'))}, 
                   lower=9, upper=10, lower_bound=7, upper_bound=15):
    best_score = 0
    best_lower = lower
    best_upper = upper
    while lower > lower_bound : 
        bands = reset_band_bounds(bands, 'Theta', 'Alpha', 'Beta', lower, upper)
        peakf = pf.PeakFrequency(nr_channels, samples, fs, bands)
        scores = peakf.fit(data.T)
        score = score_product(scores['Alpha'][0], scores['Alpha'][1])
        best_score, best_lower = swap_score(score, best_score, lower, best_lower)
        lower = lower-1
    best_score = 0      
    while upper < upper_bound : 
        bands = reset_band_bounds(bands, 'Theta', 'Alpha', 'Beta', lower, upper)
        peakf = pf.PeakFrequency(nr_channels, samples, fs, bands)
        scores = peakf.fit(data.T)
        score = score_product(scores['Alpha'][0], scores['Alpha'][1])
        best_score, best_upper = swap_score(score, best_score, upper, best_upper)
        upper = upper+1
        bands = reset_band_bounds(bands, 'Theta', 'Alpha', 'Beta', best_lower, best_upper)
    return bands

def get_mne_ready():
    return list(optimize_alpha().values())

samples = 125
bands_r = optimize_alpha(data_rhand[0,:,:125])
bands_l = optimize_alpha(data_lhand[0,:,:125])
print(bands_r)
print(bands_l)
bands = bands_l

## Topograpy Maps
*Alpha range, via trial, FM features*

In [ ]:
def plot_single_topomap(data, pos, vmin=None, vmax=None, title=None, cmap_rb=False):
    vmin = np.min(data) if vmin is None else vmin
    vmax = np.max(data) if vmax is None else vmax
    fig, ax  = plt.subplots()
    cmap = mne.viz.utils._setup_cmap('interactive', norm=1-cmap_rb)
    im, _ = mne.viz.topomap.plot_topomap(data, pos, vmin=vmin, vmax=vmax, axes=ax,
                         cmap=cmap[0], image_interp='bilinear', contours=0,
                         outlines='skirt', show=False)
    cbar, cax = mne.viz.topomap._add_colorbar(ax, im, cmap, pad=.25, title=None,
                                  size="10%", format='%3.3f')
    cbar.set_ticks((vmin, vmax))
    ax.set_title(title)
    #plt.savefig(title+".png", dpi=150)

fm = pf.PeakFrequency(nr_channels, data_lhand.shape[2], fs, bands)
acum_y = []
acum_x = []
for i in range(data_lhand.shape[0]):
    peaks_l = fm.fit(data_lhand[i,:,:].T)
    peaks_r = fm.fit(data_rhand[i,:,:].T)
    pos = pos_from_raw(raw.info, picks)
    #plot_single_topomap(peaks_l['Alpha'], pos, title = bands['Alpha'][2]+' - left, trial'+str(i), cmap_rb=True)
    #plot_single_topomap(peaks_r['Alpha'], pos, title = bands['Alpha'][2]+' - right, trial'+str(i), cmap_rb=True)
    acum_y.append(peaks_l['Alpha'])
    acum_x.append(0)
    acum_y.append(peaks_r['Alpha'])
    acum_x.append(1)

## Correlation Plots
*Alpha range*

In [ ]:
import pandas as pd

y = np.array(acum_y)
val= []
for i in range(y.shape[1]):
    corr = np.corrcoef(acum_x, y[:, i])
    val.append(np.abs(corr[0][1]))
print(val)
plot_single_topomap(val, pos, title = 'correlations', cmap_rb=True)
plt.savefig("corr-alpha.png", dpi=150)

## Time Evaluation
*!) caution, do not test the parallelized version*

In [47]:
from timeit import default_timer as timer
import peakfrequency.peak_frequency_parallel as pfpar
samples = data_lhand.shape[2]
data = data_lhand[0,:,:]
print(data.shape)
print('channels: ', nr_channels, ' samples: ', 125,' fs: ', fs, ' bands: ', bands)
for i in [125, 250, 500, 1000]:
    fm = pf.PeakFrequency(nr_channels, i, fs, bands)
    start = timer()
    fm.fit(data.T[:i])
    end = timer()
    frame = i/1000
    elapsed = end - start
    print('time for sample size: ', i, " time elapsed: ", end - start, " fraction: ", int((elapsed/frame)*1000)/10, "% of the samples time frame")





(122, 2501)
channels:  122  samples:  125  fs:  500  bands:  {'Delta': (1.0, 4.0, 'Delta (1 , 4)'), 'Theta': (4.0, 9.0, 'Theta (4.0 , 9)'), 'Alpha': (9.0, 12.0, 'Alpha (9 , 12)'), 'Beta': (12.0, 30.0, 'Beta (12 , 30.0)'), 'Gamma': (30.0, 45.0, 'Gamma (30 , 45)')}
time for sample size:  125  time elapsed:  0.027502463000018906  fraction:  22.0 % of the sample slice
time for sample size:  250  time elapsed:  0.01992861000053381  fraction:  7.9 % of the sample slice
time for sample size:  500  time elapsed:  0.08617637800034572  fraction:  17.2 % of the sample slice
time for sample size:  1000  time elapsed:  0.22272686999986036  fraction:  22.2 % of the sample slice
